In [1]:
import numpy
print(numpy.__version__)

In [2]:
import wandb
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
import os
import torch.distributed as dist
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '7030'
dist.init_process_group('gloo', rank=0, world_size=1)


In [4]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/sabl_faster_resnest50_pafpn.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/sabl_faster_resnest50_pafpn'

cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer = {'type': 'AdamW', 'lr': 2e-4, 'weight_decay': 0.001}
cfg.lr_config.step = [8,10,12]
cfg.lr_config.gamma = 0.4

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

cfg.log_config.hooks = [dict(type='TextLoggerHook'), dict(type='WandbLoggerHook')]
cfg.checkpoint_config['max_keep_ckpts']=3
cfg.evaluation.save_best = 'bbox_mAP_50'

In [5]:
print(cfg.runner)

{'type': 'EpochBasedRunner', 'max_epochs': 12}


In [6]:
model = build_detector(cfg.model)

2021-05-17 09:22:12,381 - mmdet - INFO - load model from: open-mmlab://resnest50
2021-05-17 09:22:12,383 - mmdet - INFO - Use load_from_openmmlab loader


In [7]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.80s)
creating index...
index created!


In [8]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-17 09:22:20,530 - mmdet - INFO - Start running, host: root@bc720df3e041, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/sabl_faster_resnest50_pafpn
2021-05-17 09:22:20,531 - mmdet - INFO - workflow: [('train', 1)], max: 12 epochs


Done (t=0.89s)
creating index...
index created!


/opt/ml/code/mmdetection_trash/mmdet/core/evaluation/eval_hooks.py:113: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
wandb: Currently logged in as: k1smet14 (use `wandb login --relogin` to force relogin)


2021-05-17 09:22:54,178 - mmdet - INFO - Epoch [1][50/655]	lr: 1.978e-05, eta: 1:15:06, time: 0.577, data_time: 0.050, memory: 3494, loss_rpn_cls: 0.6132, loss_rpn_bbox: 0.1455, loss_cls: 1.5768, acc: 63.6055, loss_bbox_cls: 0.1458, loss_bbox_reg: 0.1598, loss: 2.6411, grad_norm: 31.1111
2021-05-17 09:23:21,351 - mmdet - INFO - Epoch [1][100/655]	lr: 3.976e-05, eta: 1:12:27, time: 0.543, data_time: 0.007, memory: 3494, loss_rpn_cls: 0.3160, loss_rpn_bbox: 0.1017, loss_cls: 0.4251, acc: 90.7803, loss_bbox_cls: 0.2286, loss_bbox_reg: 0.2892, loss: 1.3605, grad_norm: 7.2897
2021-05-17 09:23:48,289 - mmdet - INFO - Epoch [1][150/655]	lr: 5.974e-05, eta: 1:11:04, time: 0.539, data_time: 0.007, memory: 3494, loss_rpn_cls: 0.2210, loss_rpn_bbox: 0.1079, loss_cls: 0.4646, acc: 88.7578, loss_bbox_cls: 0.2567, loss_bbox_reg: 0.3521, loss: 1.4023, grad_norm: 6.8284
2021-05-17 09:24:14,711 - mmdet - INFO - Epoch [1][200/655]	lr: 7.972e-05, eta: 1:09:49, time: 0.528, data_time: 0.007, memory: 3494,

...

2021-05-17 10:42:24,798 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.57s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.33s).
Accumulating evaluation results...
DONE (t=0.79s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.245
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.362
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.245
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.043
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.212
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.482
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.378
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.378
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.378
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]

2021-05-17 10:42:34,959 - mmdet - INFO - Now best checkpoint is epoch_12.pth.Best bbox_mAP_50 is 0.3620
2021-05-17 10:42:34,968 - mmdet - INFO - Epoch(val) [12][655]	bbox_mAP: 0.2450, bbox_mAP_50: 0.3620, bbox_mAP_75: 0.2450, bbox_mAP_s: 0.0430, bbox_mAP_m: 0.2120, bbox_mAP_l: 0.4820, bbox_mAP_copypaste: 0.245 0.362 0.245 0.043 0.212 0.482


train/loss_rpn_cls,0.00882
train/loss_rpn_bbox,0.03488
train/loss_cls,0.13301
train/acc,95.00879
train/loss_bbox_cls,0.07822
train/loss_bbox_reg,0.17576
train/loss,0.4307
train/grad_norm,4.72237
learning_rate,3e-05
momentum,0.9
_runtime,4813


train/loss_rpn_cls,█▅▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▇▅▅▄▄▅▅▃▄▃▃▃▄▃▃▄▃▃▃▃▃▃▂▂▂▃▂▂▁▂▂▁▂▁▂▁▁▁▁
train/loss_cls,▇██▇▆▆▇▆▆▅▅▆▅▆▅▆▆▅▅▄▄▄▅▄▄▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁
train/acc,▄▂▁▂▂▂▂▂▂▃▃▂▃▂▃▃▂▃▃▄▄▄▃▅▅▅▄▅▆▇▇▇▇▇█▇████
train/loss_bbox_cls,▇█▇▆▅▄▅▄▄▄▃▄▃▄▃▄▄▃▃▃▃▃▃▂▂▂▃▂▂▁▂▂▁▂▁▁▁▁▁▁
train/loss_bbox_reg,▅██▆▅▅▆▅▅▄▄▄▄▄▄▅▅▄▄▃▃▄▄▃▃▃▄▃▃▁▂▂▂▂▁▂▁▁▁▁
train/loss,██▇▆▅▅▆▅▅▄▄▄▄▄▄▄▅▄▄▃▃▃▄▃▃▃▃▂▂▂▂▂▁▂▁▁▁▁▁▁
train/grad_norm,█▇█▅▄▄▄▅▄▄▄▅▄▅▄▅▅▆▅▄▄▄▅▅▄▄▆▄▃▃▃▄▃▃▁▂▂▂▁▂
learning_rate,▁▅█████████████████████████▃▃▃▃▃▃▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
